In [98]:
## 加载数据
from sklearn.datasets import load_iris
import numpy as np
from collections import Counter
from sklearn.model_selection import StratifiedKFold 
from sklearn.preprocessing import OneHotEncoder
data = load_iris()
train_data = data.data
train_y = data.target
data = np.hstack((train_data,train_y.reshape(-1,1)))
np.random.shuffle(data)
kflod = StratifiedKFold(n_splits = 10,shuffle = True, random_state = 42)
data.shape

(150, 5)

In [99]:
import tensorflow as tf
dataset_size = data.shape[0]
batch_size = 10
w1 = tf.Variable(tf.random_normal([4, 100], stddev = 1, seed = 1))
b1 = tf.Variable(tf.random_normal([100,1], stddev = 1, seed = 1))
w2 = tf.Variable(tf.random_normal([100, 3], stddev = 1, seed = 1))
# 指定大小为None的话，可以随意指定batch_size 的大小
x = tf.placeholder(tf.float32, shape = (None, 4), name = "x-input")
y_ = tf.placeholder(tf.float32, shape = (None, 3), name ="y-input")

In [100]:
# 定义要执行的步骤
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)
pred = tf.nn.softmax(y)

In [101]:
# 定义要优化的目标
cross_entropy = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y))
train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
acc = tf.reduce_mean(tf.cast(tf.equal(tf.one_hot(tf.argmax(pred,axis = 1), depth = 3),y_), tf.float32))

- 交叉熵损失函数
$$h = -\sum_{i=1}^{n}y_{i} * log(y_{i}^{'})$$ 其中$y_{i}$表示预测的第i个分量

In [104]:
one_hot = OneHotEncoder()
loss = []
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    for train_index, test_index in kflod.split(data[:,:-1],data[:,-1]):
        train_data = data[train_index]
        test_data = data[test_index]
        Step = 100
        for i in range(Step):
            start = (i * batch_size) % dataset_size
            end = min(start + batch_size, dataset_size)
            label = tf.one_hot(train_data[start: end,-1],depth = 3)
            label = sess.run(label)
            sess.run(train_step, feed_dict = {x: train_data[start: end,:-1], y_ : label})
#             if i % 1000 == 0:
#                 total_cross_entropy = sess.run(cross_entropy, feed_dict = {x: data[start: end,:-1], y_: data[start:end,-1].reshape(-1,1)})
#                 loss.append(total_cross_entropy)
#                 print("after %d steps ,cross entropy on all data is %g"%(i, total_cross_entropy))
        test_label = tf.one_hot(test_data[:,-1],depth = 3)
        test_label = sess.run(test_label)
        print(sess.run(acc, feed_dict = {x : test_data[:,:-1],y_ : test_label}))


0.95555556
0.95555556
0.9111111
0.9111111
1.0
0.95555556
1.0
1.0
0.95555556
0.9111111


In [13]:
# 复习一下tf中的简单的变量定义还有placeholder的定义
import tensorflow as tf
v = tf.Variable(tf.random_normal([2,3], stddev = 1,seed = 1))
x = tf.placeholder(tf.float32, shape = (2,1), name = 'x_input')
y = tf.placeholder(tf.float32, shape = (2,2), name = 'y_input')
w = tf.matmul(y,x)
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run(v))
    print(sess.run(w, feed_dict = {x: [[1],[2]],y: [[1,2],[3,4]]}))

[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[ 5.]
 [11.]]


In [23]:
## tf.clip_by_value(x,min,max)
clip_value = tf.placeholder(tf.int32, shape = [1], name = 'x')
with tf.Session() as sess:
    for i in range(20):
        cliped = tf.clip_by_value(clip_value, 5,10)
        print(sess.run(cliped ,feed_dict = {clip_value : [i]}))

In [28]:
## tf.reduce_mean(tensor,axis = 0) 0 为列，而且算均值的时候是看数据类型的
# v = tf.Variable(shape = (1,))
with tf.Session() as sess:
    print(sess.run(tf.reduce_mean([0.0,1.0,0.0,1.0])))

0.5


In [37]:
v1 = tf.constant([1,0])
pred = tf.constant([[0.4,0.6],[0.6,0.4]])
with tf.Session() as sess:
    print(sess.run(tf.nn.softmax_cross_entropy_with_logits(labels = v1, logits = pred)))

[0.79813886 0.59813887]


In [105]:
# tf.reduce_sum() 应该是求所有的和
with tf.Session() as sess:
    v = tf.constant([1,2,3])
    print(sess.run(tf.reduce_sum(v)))# output: [3 x 3]

6
